In [1]:
import os
import sys
os.environ['SPARK_HOME']='/home/cloudera/spark230hadoop26'
os.environ['PYLIB']=os.environ['SPARK_HOME']+'/python/lib'
sys.path.insert(0,os.environ['PYLIB']+'/py4j-0.10.6-src.zip')
sys.path.insert(1,os.environ['PYLIB']+'/pyspark.zip')

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [3]:
spark = SparkSession.builder.appName('TestHive') \
.config('spark.warehouse.dir','/apps/hive/warehouse') \
.enableHiveSupport().getOrCreate()

In [4]:
# Having set the driver and driver options we should have spark representing spark session 
# available straight away
spark.version

'2.3.0'

In [50]:
from pyspark.sql.functions import *

In [51]:
cm_df = spark.read.option(
    "inferSchema", True).option(
    "header", True).csv(
    "hdfs://localhost:8020/user/cloudera/findata/cm")

In [31]:
cm_df.show(2)

+----------+------+----+----+-----+-----+----+---------+---------+-------------+-----------+-----------+------------+----+
|    SYMBOL|SERIES|OPEN|HIGH|  LOW|CLOSE|LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|_c13|
+----------+------+----+----+-----+-----+----+---------+---------+-------------+-----------+-----------+------------+----+
| 20MICRONS|    EQ|37.8|38.9|36.55|38.05|38.2|     37.8|   182096|    6917530.0|17-OCT-2016|        877|INE144J01027|null|
|3IINFOTECH|    EQ| 5.8| 6.1|  5.8|  6.0| 6.1|      5.8|  2345944|1.395231735E7|17-OCT-2016|        993|INE748C01020|null|
+----------+------+----+----+-----+-----+----+---------+---------+-------------+-----------+-----------+------------+----+
only showing top 2 rows



In [32]:
cmdf = cm_df.drop("_c13")
print("The cash market data frame schema")
cmdf.printSchema()

The cash market data frame schema
root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)



In [33]:
fo_df = spark.read.option(
    "inferSchema", True).option(
    "header", True).csv(
    "hdfs://localhost:8020/user/cloudera/findata/fo")

In [34]:
fodf = fo_df.drop("_c15")

In [55]:
def mnameToNo(dt):
    mname = dt[3:6]
    calendar = {"JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
      "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08", "SEP": "09", "OCT": "10",
      "NOV": "11", "DEC": "12"}
    return dt.replace(mname, calendar[mname])

In [56]:
mnameToNo('12-JAN-2016')

'12-01-2016'

In [57]:
udf_mname_to_no = udf(mnameToNo)

In [58]:
fodf.show(2)

+----------+---------+-----------+---------+----------+-------+-------+-------+--------+---------+---------+----------+--------+---------+-----------+
|INSTRUMENT|   SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|   OPEN|   HIGH|    LOW|   CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|
+----------+---------+-----------+---------+----------+-------+-------+-------+--------+---------+---------+----------+--------+---------+-----------+
|    FUTIDX|BANKNIFTY|24-Nov-2016|      0.0|        XX|18460.0|18466.8|18229.6|18263.95|  18256.1|   109639| 804002.68| 1330360|  -713040|24-NOV-2016|
|    FUTIDX|BANKNIFTY|29-Dec-2016|      0.0|        XX|18541.0|18563.5|18352.0|18392.25| 18392.25|    68132| 502565.82| 2088480|  1132560|24-NOV-2016|
+----------+---------+-----------+---------+----------+-------+-------+-------+--------+---------+---------+----------+--------+---------+-----------+
only showing top 2 rows



In [61]:
print("verifying that we our udf works and we can plug in to_timestamp")
fodf.select(to_timestamp(udf_mname_to_no("TIMESTAMP"), "dd-MM-yyyy").alias('rts')).show(2)

verifying that we our udf works and we can plug in to_timestamp
+-------------------+
|                rts|
+-------------------+
|2016-11-24 00:00:00|
|2016-11-24 00:00:00|
+-------------------+
only showing top 2 rows



In [62]:
# register a table against it
fodf.createOrReplaceTempView('fut_data')

In [63]:
print("taking a look at settlement date data")
spark.sql("select * from fut_data where lower(expiry_dt) = lower(timestamp)limit 5").show(3)

taking a look at settlement date data
+----------+----------+-----------+---------+----------+-------+-------+-------+--------+---------+---------+----------+--------+---------+-----------+
|INSTRUMENT|    SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|   OPEN|   HIGH|    LOW|   CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|
+----------+----------+-----------+---------+----------+-------+-------+-------+--------+---------+---------+----------+--------+---------+-----------+
|    FUTIDX| BANKNIFTY|24-Nov-2016|      0.0|        XX|18460.0|18466.8|18229.6|18263.95|  18256.1|   109639| 804002.68| 1330360|  -713040|24-NOV-2016|
|    FUTIDX|     NIFTY|24-Nov-2016|      0.0|        XX| 7987.0|8018.65|7947.25|  7968.2|   7965.5|   218779|1309794.07| 9314925| -3437550|24-NOV-2016|
|    FUTIDX|NIFTYINFRA|24-Nov-2016|      0.0|        XX|    0.0|    0.0|    0.0| 2947.85|   2671.3|        0|       0.0|       0|        0|24-NOV-2016|
+----------+----------+-----------+---------+-----

In [65]:
print("settlement data look using df api")
fodf.where("lower(expiry_dt) = lower(timestamp)").limit(5).show()

settlement data look using df api
+----------+----------+-----------+---------+----------+-------+-------+-------+--------+---------+---------+----------+--------+---------+-----------+
|INSTRUMENT|    SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|   OPEN|   HIGH|    LOW|   CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|
+----------+----------+-----------+---------+----------+-------+-------+-------+--------+---------+---------+----------+--------+---------+-----------+
|    FUTIDX| BANKNIFTY|24-Nov-2016|      0.0|        XX|18460.0|18466.8|18229.6|18263.95|  18256.1|   109639| 804002.68| 1330360|  -713040|24-NOV-2016|
|    FUTIDX|     NIFTY|24-Nov-2016|      0.0|        XX| 7987.0|8018.65|7947.25|  7968.2|   7965.5|   218779|1309794.07| 9314925| -3437550|24-NOV-2016|
|    FUTIDX|NIFTYINFRA|24-Nov-2016|      0.0|        XX|    0.0|    0.0|    0.0| 2947.85|   2671.3|        0|       0.0|       0|        0|24-NOV-2016|
|    FUTIDX|   NIFTYIT|24-Nov-2016|      0.0|        X

## F&O Futures Net PL estimation
## Lets say we have the following trades for 3/4 days
### Day  Symbol Long/Short Qty Price  TotalOutlay  OI    CHG_OI
###  1         X                Buy         100   20      2000           100   100
###  2          X               Buy            50   30     3500            150     50
###  3          X                Sell         100    10     2500            50      -100
### Lets say close price on the third day was 12. Then further 12 x 50 needs to be deducted to find out the net settlement amount. 
### So we would have 2500 - 600 - a profit of 1900 made by the sellers

## Lets check a profitable scenario for the buyers
### Day  Symbol Long/Short Qty Price  TotalOutlay  OI    CHG_OI
###  1         X                Buy         100   20      2000           100   100
###  2          X               Buy            50   30     3500            150     50
###  3          X                Sell         100    40     -500            50      -100
### Lets say close price on the third day was 42. Then further 42 x 50 needs to be deducted to find out the net settlement amount. 
### So we would have -500-2100 - a profit of 2600 made by the buyers



In [67]:
spark.udf.register("udf_mname_to_no", mnameToNo)

print("Using sql to show overall profit loss status for f&o market")
spark.sql("""
select timestamp, symbol,expiry_dt,chg_in_oi,close,open_int,
sum(chg_in_oi)
over (partition by symbol,expiry_dt order by to_timestamp(udf_mname_to_no(timestamp), "dd-MM-yyyy")) as cum_chg_oi,
sum(chg_in_oi*close)
over ( partition by symbol,expiry_dt order by to_timestamp(udf_mname_to_no(timestamp), "dd-MM-yyyy"))
as b_s_pl_status
from fut_data
where instrument like 'FUT%' and symbol = 'INFY'
order by to_timestamp(udf_mname_to_no(timestamp), "dd-MM-yyyy")
""").show(50, False)

Using sql to show overall profit loss status for f&o market
+-----------+------+-----------+---------+-------+--------+----------+---------------+
|timestamp  |symbol|expiry_dt  |chg_in_oi|close  |open_int|cum_chg_oi|b_s_pl_status  |
+-----------+------+-----------+---------+-------+--------+----------+---------------+
|01-OCT-2013|INFY  |26-Dec-2013|500      |3073.8 |875     |500       |1536900.0      |
|01-OCT-2013|INFY  |31-Oct-2013|52125    |3035.25|2332625 |52125     |1.5821240625E8 |
|01-OCT-2013|INFY  |28-Nov-2013|1750     |3055.75|24250   |1750      |5347562.5      |
|03-OCT-2013|INFY  |28-Nov-2013|2500     |3074.45|26750   |4250      |1.30336875E7   |
|03-OCT-2013|INFY  |31-Oct-2013|314875   |3057.1 |2647500 |367000    |1.12081676875E9|
|03-OCT-2013|INFY  |26-Dec-2013|4750     |3096.35|5625    |5250      |1.62445625E7   |
|04-OCT-2013|INFY  |28-Nov-2013|10250    |3048.2 |37000   |14500     |4.42777375E7   |
|04-OCT-2013|INFY  |26-Dec-2013|125      |3069.5 |5750    |5375      |

In [70]:
# create a view with the proper timestamp column - tsp -timestamp proper
fodfvw = fodf.withColumn("tsp", to_timestamp(udf_mname_to_no("TIMESTAMP"), "dd-MM-yyyy"))

In [84]:
from pyspark.sql import Window
partitionWindow = Window.partitionBy("symbol", "expiry_dt").orderBy("tsp")
#then define the aggregation functions we are interested in over it
sumChgoi = sum("CHG_IN_OI").over(partitionWindow)
plStatus = sum(col('CHG_IN_OI') * col('close')).over(partitionWindow)

In [88]:
# and plug in the regular dataframe api
print("window functions executed using df api")
fodfvw.select("symbol", "tsp", "timestamp",
    "expiry_dt", "chg_in_oi", "open_int", "close",
    sumChgoi.alias("oitrack"), plStatus.alias("plstatus")).filter(
    "symbol = 'INFY' and instrument like 'FUT%'").orderBy(
    to_timestamp(udf_mname_to_no('timestamp'), "dd-MM-yyyy")).show(20, False)

window functions executed using df api
+------+-------------------+-----------+-----------+---------+--------+-------+--------+---------------+
|symbol|tsp                |timestamp  |expiry_dt  |chg_in_oi|open_int|close  |oitrack |plstatus       |
+------+-------------------+-----------+-----------+---------+--------+-------+--------+---------------+
|INFY  |2013-10-01 00:00:00|01-OCT-2013|28-Nov-2013|1750     |24250   |3055.75|7625    |6548075.0      |
|INFY  |2013-10-01 00:00:00|01-OCT-2013|31-Oct-2013|52125    |2332625 |3035.25|372750  |1.917483375E8  |
|INFY  |2013-10-01 00:00:00|01-OCT-2013|26-Dec-2013|500      |875     |3073.8 |500     |1536900.0      |
|INFY  |2013-10-03 00:00:00|03-OCT-2013|28-Nov-2013|2500     |26750   |3074.45|18125   |1.587411875E7  |
|INFY  |2013-10-03 00:00:00|03-OCT-2013|31-Oct-2013|314875   |2647500 |3057.1 |1167000 |1.19958955625E9|
|INFY  |2013-10-03 00:00:00|03-OCT-2013|26-Dec-2013|4750     |5625    |3096.35|5250    |1.62445625E7   |
|INFY  |2013-10-

In [89]:
# here we check the status on the expiry dates for the pl situation
print("status on the expiry dates for the pl situation")
spark.sql("""
select f.* from
(
select timestamp, symbol,expiry_dt,chg_in_oi,close,open_int,
sum(chg_in_oi) over (partition by symbol,expiry_dt order by to_timestamp(udf_mname_to_no(timestamp), "dd-MM-yyyy")) as cum_chg_oi,
sum(chg_in_oi*close) over ( partition by symbol,expiry_dt order by to_timestamp(udf_mname_to_no(timestamp), "dd-MM-yyyy"))
as b_s_pl_status
from fut_data
where instrument like 'FUT%'
) f
where lower(f.expiry_dt) = lower(f.timestamp) and symbol = 'INFY'
order by to_timestamp(udf_mname_to_no(f.timestamp), "dd-MM-yyyy")
""").show(50, False)

status on the expiry dates for the pl situation
+-----------+------+-----------+---------+-------+--------+----------+--------------------+
|timestamp  |symbol|expiry_dt  |chg_in_oi|close  |open_int|cum_chg_oi|b_s_pl_status       |
+-----------+------+-----------+---------+-------+--------+----------+--------------------+
|31-OCT-2013|INFY  |31-Oct-2013|-405250  |3309.05|1157125 |-1311625  |-4.49664064375E9    |
|28-NOV-2013|INFY  |28-Nov-2013|-433375  |3323.5 |560000  |526750    |1.79817521875E9     |
|26-DEC-2013|INFY  |26-Dec-2013|-478875  |3517.05|1063000 |1062250   |3.2057007125E9      |
|30-OCT-2014|INFY  |30-Oct-2014|-454500  |3946.4 |1602500 |-1780375  |-7.0114377125E9     |
|27-NOV-2014|INFY  |27-Nov-2014|-349250  |4362.3 |1134125 |1113500   |3.4529359375E9      |
|24-DEC-2014|INFY  |24-Dec-2014|-1330000 |1933.05|4312250 |-615375   |1.00962228875E10    |
|29-OCT-2015|INFY  |29-Oct-2015|-1551750 |1144.85|2279000 |-10539250 |-1.20626163125E10   |
|26-NOV-2015|INFY  |26-Nov-2015|

In [91]:
# create a proper timestamp column for the cash market data
cmdfvw = cmdf.withColumn(
    "tsp",
    to_timestamp(udf_mname_to_no("TIMESTAMP"), "dd-MM-yyyy"))

cmdfvw.createOrReplaceTempView("cmdata")

print("verifying the extra column we added as a proper timestamp")
cmdfvw.limit(5).show()

verifying the extra column we added as a proper timestamp
+----------+------+-------+-------+--------+-------+-------+---------+---------+-------------+-----------+-----------+------------+-------------------+
|    SYMBOL|SERIES|   OPEN|   HIGH|     LOW|  CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|                tsp|
+----------+------+-------+-------+--------+-------+-------+---------+---------+-------------+-----------+-----------+------------+-------------------+
| 20MICRONS|    EQ|   37.8|   38.9|   36.55|  38.05|   38.2|     37.8|   182096|    6917530.0|17-OCT-2016|        877|INE144J01027|2016-10-17 00:00:00|
|3IINFOTECH|    EQ|    5.8|    6.1|     5.8|    6.0|    6.1|      5.8|  2345944|1.395231735E7|17-OCT-2016|        993|INE748C01020|2016-10-17 00:00:00|
|   3MINDIA|    EQ|13095.0|13099.0|12964.05|12976.5|12980.0|  12992.9|      613|    7976511.3|17-OCT-2016|        130|INE470A01017|2016-10-17 00:00:00|
|   8KMILES|    EQ|  790.0|  7

In [92]:
# use sql to carry out window functions moving average using rows preceding
print("using sql to carry out window functions moving average using rows preceding")
spark.sql("""
select symbol, timestamp,close,
avg(close) over(partition by symbol order by tsp rows 5 preceding )as mv5avg,
avg(close) over(partition by symbol order by tsp rows 20 preceding )as mv20avg
from cmdata
where symbol = 'INFY'
order by tsp
""").show()

using sql to carry out window functions moving average using rows preceding
+------+-----------+-------+------------------+------------------+
|symbol|  timestamp|  close|            mv5avg|           mv20avg|
+------+-----------+-------+------------------+------------------+
|  INFY|01-JAN-2016|1105.25|           1105.25|           1105.25|
|  INFY|04-JAN-2016| 1078.9|          1092.075|          1092.075|
|  INFY|05-JAN-2016|1074.05|1086.0666666666666|1086.0666666666666|
|  INFY|06-JAN-2016|1069.35|1081.8874999999998|1081.8874999999998|
|  INFY|07-JAN-2016| 1050.8|1075.6699999999998|1075.6699999999998|
|  INFY|08-JAN-2016| 1063.3|1073.6083333333333|1073.6083333333333|
|  INFY|11-JAN-2016| 1055.7|           1065.35|           1071.05|
|  INFY|12-JAN-2016|1049.95|1060.5249999999999|         1068.4125|
|  INFY|13-JAN-2016| 1083.4|1062.0833333333333|1070.0777777777776|
|  INFY|14-JAN-2016| 1133.0|1072.6916666666666|           1076.37|
|  INFY|15-JAN-2016| 1139.9|1087.5416666666667|1082.1

In [95]:
# using dataframe api to calculate moving average
print("using dataframe api to calculate moving average")
mvngAvgSpec = avg("close").over(
    Window.partitionBy("symbol").orderBy("tsp").rowsBetween(-3, 0))
 
cmdfvw.select("symbol", "timestamp", "tsp", "close",
    mvngAvgSpec.alias("mvng_avg_5")).filter(
    "symbol = 'INFY'").orderBy(
    "tsp").show()


using dataframe api to calculate moving average
+------+-----------+-------------------+-------+------------------+
|symbol|  timestamp|                tsp|  close|        mvng_avg_5|
+------+-----------+-------------------+-------+------------------+
|  INFY|01-JAN-2016|2016-01-01 00:00:00|1105.25|           1105.25|
|  INFY|04-JAN-2016|2016-01-04 00:00:00| 1078.9|          1092.075|
|  INFY|05-JAN-2016|2016-01-05 00:00:00|1074.05|1086.0666666666666|
|  INFY|06-JAN-2016|2016-01-06 00:00:00|1069.35|1081.8874999999998|
|  INFY|07-JAN-2016|2016-01-07 00:00:00| 1050.8|1068.2749999999999|
|  INFY|08-JAN-2016|2016-01-08 00:00:00| 1063.3|          1064.375|
|  INFY|11-JAN-2016|2016-01-11 00:00:00| 1055.7|         1059.7875|
|  INFY|12-JAN-2016|2016-01-12 00:00:00|1049.95|         1054.9375|
|  INFY|13-JAN-2016|2016-01-13 00:00:00| 1083.4|         1063.0875|
|  INFY|14-JAN-2016|2016-01-14 00:00:00| 1133.0|         1080.5125|
|  INFY|15-JAN-2016|2016-01-15 00:00:00| 1139.9|         1101.5625|


In [96]:
# exercise - add on 20 days moving average

# using row number

print("using row number with sql")

spark.sql("""
select symbol, timestamp, close,
row_number() over(partition by symbol order by tsp) as rno
from cmdata
where symbol in ('INFY', 'TCS')""").show()


using row number with sql
+------+-----------+-------+---+
|symbol|  timestamp|  close|rno|
+------+-----------+-------+---+
|  INFY|01-JAN-2016|1105.25|  1|
|  INFY|04-JAN-2016| 1078.9|  2|
|  INFY|05-JAN-2016|1074.05|  3|
|  INFY|06-JAN-2016|1069.35|  4|
|  INFY|07-JAN-2016| 1050.8|  5|
|  INFY|08-JAN-2016| 1063.3|  6|
|  INFY|11-JAN-2016| 1055.7|  7|
|  INFY|12-JAN-2016|1049.95|  8|
|  INFY|13-JAN-2016| 1083.4|  9|
|  INFY|14-JAN-2016| 1133.0| 10|
|  INFY|15-JAN-2016| 1139.9| 11|
|  INFY|18-JAN-2016| 1131.9| 12|
|  INFY|19-JAN-2016|1139.65| 13|
|  INFY|20-JAN-2016|1121.25| 14|
|  INFY|21-JAN-2016|1137.05| 15|
|  INFY|22-JAN-2016|1136.25| 16|
|  INFY|25-JAN-2016|1137.65| 17|
|  INFY|27-JAN-2016| 1138.5| 18|
|  INFY|28-JAN-2016| 1129.6| 19|
|  INFY|29-JAN-2016| 1164.9| 20|
+------+-----------+-------+---+
only showing top 20 rows



In [97]:
# dafaframe api using row number
print("dafaframe api showing row number")
rnospec = row_number().over(
    Window.partitionBy("symbol").orderBy(
    "tsp"))

cmdfvw.select("symbol", "timestamp", "close",
    rnospec.alias("rno")).filter(
    "symbol in ('INFY','TCS')").show()

dafaframe api showing row number
+------+-----------+-------+---+
|symbol|  timestamp|  close|rno|
+------+-----------+-------+---+
|  INFY|01-JAN-2016|1105.25|  1|
|  INFY|04-JAN-2016| 1078.9|  2|
|  INFY|05-JAN-2016|1074.05|  3|
|  INFY|06-JAN-2016|1069.35|  4|
|  INFY|07-JAN-2016| 1050.8|  5|
|  INFY|08-JAN-2016| 1063.3|  6|
|  INFY|11-JAN-2016| 1055.7|  7|
|  INFY|12-JAN-2016|1049.95|  8|
|  INFY|13-JAN-2016| 1083.4|  9|
|  INFY|14-JAN-2016| 1133.0| 10|
|  INFY|15-JAN-2016| 1139.9| 11|
|  INFY|18-JAN-2016| 1131.9| 12|
|  INFY|19-JAN-2016|1139.65| 13|
|  INFY|20-JAN-2016|1121.25| 14|
|  INFY|21-JAN-2016|1137.05| 15|
|  INFY|22-JAN-2016|1136.25| 16|
|  INFY|25-JAN-2016|1137.65| 17|
|  INFY|27-JAN-2016| 1138.5| 18|
|  INFY|28-JAN-2016| 1129.6| 19|
|  INFY|29-JAN-2016| 1164.9| 20|
+------+-----------+-------+---+
only showing top 20 rows



In [98]:
# creating a view that we will use to cube by year, month, day
cmdfymdvw = cmdfvw.withColumn(
    "yr", year("tsp")).withColumn(
    "mnth", month("tsp")).withColumn(
    "dy", dayofmonth("tsp"))

In [99]:
cmdfymdvw.createOrReplaceTempView("cb_tbl")

In [100]:
# grouping sets using sql
print("grouping sets using sql")
spark.sql("""
select symbol, yr, mnth, dy , avg(close) as avgcls, avg(tottrdqty) as avgqty, avg(tottrdval) as avgval from cb_tbl
where symbol in ('INFY', 'TCS')
group by symbol, yr, mnth, dy
grouping sets((yr), (yr, mnth))
""").show()

grouping sets using sql
+------+----+----+----+------------------+------------------+--------------------+
|symbol|  yr|mnth|  dy|            avgcls|            avgqty|              avgval|
+------+----+----+----+------------------+------------------+--------------------+
|  null|2016|   4|null|1828.4243243243243|2447373.4594594594| 3.814428185441892E9|
|  null|2016|null|null| 1759.183367556468|2264549.0985626285| 3.210636273691889E9|
|  null|2016|   1|null|1716.7112499999998|       2548902.675|  3.56202842181875E9|
|  null|2016|  12|null|1620.5099999999998|         1904487.8|2.5012165174449997E9|
|  null|2016|  11|null| 1585.217857142857| 2626080.738095238|3.3161159360845237E9|
|  null|2016|   9|null|1725.6512500000001|       2133414.525|  3.10294392830375E9|
|  null|2016|   6|null|1894.8690476190477|2003678.5238095238| 3.098018789891667E9|
|  null|2016|   5|null|1869.1511363636362|1635787.9318181819|2.4878617885590906E9|
|  null|2016|   3|null|           1783.45|       2507484.225|  

In [101]:
#cube using sql
print("cube using sql")
spark.sql("""
select symbol,yr,mnth,dy,
avg(close) as avgcls,
avg(tottrdqty) as avgqty,
avg(tottrdval) as avgval
from cb_tbl
group by symbol,yr,mnth,dy with cube
order by yr ,mnth""").show()

cube using sql
+----------+----+----+----+------------------+------------------+--------------------+
|    symbol|  yr|mnth|  dy|            avgcls|            avgqty|              avgval|
+----------+----+----+----+------------------+------------------+--------------------+
|      STAR|null|null|null|1070.5236625514403|414849.65432098764| 4.361414973613168E8|
|HEXATRADEX|null|null|  20|            16.125|          8343.125|        139597.83125|
|  SUNDARAM|null|null|null|3.5483539094650203| 809838.7530864198|  3392860.3639917686|
|    RPOWER|null|null|  17|47.128571428571426|2986257.8571428573| 1.404933790357143E8|
|TIPSINDLTD|null|null|null| 63.72325102880659|13911.024691358025|    944007.320164609|
|     ABFRL|null|null|null|146.75089285714287| 833555.6607142857|1.2638451688214283E8|
|    VIPIND|null|null|null|116.42489711934154|433437.25514403294|  5.23858155409465E7|
|      CAPF|null|null|  17| 549.3428571428572| 161714.2857142857| 9.588909385714285E7|
|VIVIMEDLAB|null|null|  17| 

In [103]:
# cube using dafaframe api
print("cube using dafaframe api")
cmdfymdvw.cube("symbol", "yr", "mnth", "dy").agg(
      avg("close").alias("avgcls"),
      avg("tottrdqty").alias("avgqty"),
      avg("tottrdval").alias("tottrdval")).show()

cube using dafaframe api
+----------+----+----+---+--------+-----------+--------------+
|    symbol|  yr|mnth| dy|  avgcls|     avgqty|     tottrdval|
+----------+----+----+---+--------+-----------+--------------+
|       ADI|2016|  10| 17|   493.3|     9548.0|    4673864.75|
|BLUESTARCO|2016|  10| 17|  574.15|    66800.0| 3.860295415E7|
|    CONCOR|2016|  10| 17|  1321.4|    91679.0| 1.227066093E8|
|     CREST|2016|  10| 17|    63.5|     4286.0|     272480.45|
| MUKANDLTD|2016|  10| 17|  33.725|    18937.0|   1208659.925|
|SHRIRAMCIT|2016|  10| 17|1431.446|     3188.2|   7252266.686|
|   SPENTEX|2016|  10| 17|    7.25|    34466.0|     244910.75|
|THANGAMAYL|2016|  10| 17|  238.75|    15200.0|    3610708.15|
|   VOLTAMP|2016|  10| 17|   798.4|    13919.0|   1.1186215E7|
|  DECCANCE|2016|  10| 19| 1156.55|     8495.0|     9951963.6|
|    ECLERX|2016|  10| 19| 1632.35|    57633.0|  9.42413545E7|
|INDIAGLYCO|2016|  10| 19|   135.5|   195117.0|  2.66427313E7|
|      VEDL|2016|  10| 19|   1

In [107]:
# cube - see the overall aggregates
print("cube - see the overall aggregates")
cmdfymdvw.cube("symbol", "yr", "mnth", "dy"). agg(
      avg("close").alias("avgcls"),
      avg("tottrdqty").alias("avgqty"),
      avg("tottrdval").alias("tottrdval")).filter(
    isnull("symbol")).show()

cube - see the overall aggregates
+------+----+----+---+------------------+------------------+--------------------+
|symbol|  yr|mnth| dy|            avgcls|            avgqty|           tottrdval|
+------+----+----+---+------------------+------------------+--------------------+
|  null|null|null| 15| 498.0799863945573| 634404.2633408919|1.2038941464620414E8|
|  null|2016|   7| 26| 512.8960454002394| 811648.6666666666| 1.404206025329032E8|
|  null|null|  11| 23| 512.5913321385904|514992.37096774194|1.1107890112371586E8|
|  null|null|   3|  8|473.69639902676397| 659031.6198296837|1.0881725103006685E8|
|  null|2016|   4| 26| 498.2777498467199| 533269.0331085224| 1.120971077546228E8|
|  null|null|  12| 23| 501.8199111374399| 432640.1623222749| 9.043732604127386E7|
|  null|2016|  12|  5|  510.057899408284| 401353.9514792899| 8.532174424066271E7|
|  null|2016|null| 28|502.73386404652086| 632680.0222578704|1.2539030228138427E8|
|  null|2016|   7| 28| 513.5367046135411| 920008.1449970042|1.79

In [108]:
# rolling using sql
print("rolling using sql")
spark.sql("""
select symbol,yr,mnth,dy,
avg(close) as avgcls, avg(tottrdqty) as avgqty,
avg(tottrdval) as avgval from cb_tbl
where symbol = 'INFY' and mnth = 1
group by symbol,yr,mnth,dy with rollup
order by yr ,mnth, dy""").show()

rolling using sql
+------+----+----+----+---------+-----------+-----------------+
|symbol|  yr|mnth|  dy|   avgcls|     avgqty|           avgval|
+------+----+----+----+---------+-----------+-----------------+
|  INFY|null|null|null|1107.0175|  3913439.3|4.3725684664475E9|
|  null|null|null|null|1107.0175|  3913439.3|4.3725684664475E9|
|  INFY|2016|null|null|1107.0175|  3913439.3|4.3725684664475E9|
|  INFY|2016|   1|null|1107.0175|  3913439.3|4.3725684664475E9|
|  INFY|2016|   1|   1|  1105.25|   903275.0|   9.9643325865E8|
|  INFY|2016|   1|   4|   1078.9|  1987681.0|  2.16223078375E9|
|  INFY|2016|   1|   5|  1074.05|  2474893.0|  2.65597546075E9|
|  INFY|2016|   1|   6|  1069.35|  2794164.0|   2.9829102378E9|
|  INFY|2016|   1|   7|   1050.8|  2647044.0|   2.7992557204E9|
|  INFY|2016|   1|   8|   1063.3|  1578358.0|  1.67552198635E9|
|  INFY|2016|   1|  11|   1055.7|  2680311.0|   2.8350194513E9|
|  INFY|2016|   1|  12|  1049.95|  3282606.0|  3.43550012205E9|
|  INFY|2016|   1|  13

In [130]:
# Here we take a look at cumulative, rolling profit
# status for buyers and sellers
# We can consider a positive value to indicate that buyers made a profit
# as the sale price was more than the buy one
# and vice versa that the short side was in profit if we have a negative figure
spark.sql('select symbol, instrument, expiry_dt, timestamp, chg_in_oi, \
        sum(chg_in_oi) over ( \
        partition by symbol,expiry_dt \
        order by udf_mname_to_no(timestamp) \
        ) as cum_chg_oi,\
        sum(chg_in_oi * close) over ( \
        partition by symbol,expiry_dt \
        order by udf_mname_to_no(timestamp) \
        ) as b_s_pl_status \
          from fut_data ').filter(
    "symbol = 'INFY' and expiry_dt = '26-Feb-2015' \
    and chg_in_oi > 0 and instrument='FUTSTK'").orderBy('timestamp').show(300, False)

+------+----------+-----------+-----------+---------+----------+--------------+
|symbol|instrument|expiry_dt  |timestamp  |chg_in_oi|cum_chg_oi|b_s_pl_status |
+------+----------+-----------+-----------+---------+----------+--------------+
|INFY  |FUTSTK    |26-Feb-2015|01-DEC-2014|9000     |9000      |3.93615E7     |
|INFY  |FUTSTK    |26-Feb-2015|02-DEC-2014|250      |9250      |3.99044875E7  |
|INFY  |FUTSTK    |26-Feb-2015|03-DEC-2014|1250     |10500     |4.25919875E7  |
|INFY  |FUTSTK    |26-Feb-2015|08-DEC-2014|1750     |19000     |4.4271175E7   |
|INFY  |FUTSTK    |26-Feb-2015|09-DEC-2014|1000     |20500     |4.62811375E7  |
|INFY  |FUTSTK    |26-Feb-2015|10-DEC-2014|10250    |32750     |6.69580375E7  |
|INFY  |FUTSTK    |26-Feb-2015|11-DEC-2014|250      |34500     |6.74921625E7  |
|INFY  |FUTSTK    |26-Feb-2015|12-DEC-2014|500      |35250     |6.84884625E7  |
|INFY  |FUTSTK    |26-Feb-2015|15-DEC-2014|500      |36000     |6.9472975E7   |
|INFY  |FUTSTK    |26-Feb-2015|17-DEC-20

In [133]:
spark.sql("select substring('10-jan-2014',4, 3)").show()

+----------------------------+
|substring(10-jan-2014, 4, 3)|
+----------------------------+
|                         jan|
+----------------------------+



In [140]:
spark.sql('select f.* from \
( \
        select symbol,expiry_dt,timestamp,chg_in_oi, \
        sum(chg_in_oi) over ( \
        partition by symbol,expiry_dt \
        order by udf_mname_to_no(timestamp) \
        ) as cum_chg_oi,\
        open_int, \
        sum(chg_in_oi*close) over ( \
        partition by symbol,expiry_dt \
        order by udf_mname_to_no(timestamp) \
        ) - open_int * close  as b_s_pl_status \
          from fut_data \
 ) f where lower(f.timestamp) = lower(f.expiry_dt) ').show()

+------+-----------+-----------+---------+----------+--------+-------------+
|symbol|  expiry_dt|  timestamp|chg_in_oi|cum_chg_oi|open_int|b_s_pl_status|
+------+-----------+-----------+---------+----------+--------+-------------+
|  ALBK|24-Dec-2014|24-DEC-2014| -3018000|  -2972000| 1372000|  -9.485959E8|
|  ALBK|24-Dec-2014|24-DEC-2014|        0|  -2972000|       0|  -7.772331E8|
|  ALBK|24-Dec-2014|24-DEC-2014|        0|  -2972000|       0|  -7.772331E8|
|  ALBK|24-Dec-2014|24-DEC-2014|        0|  -2972000|       0|  -7.772331E8|
|  ALBK|24-Dec-2014|24-DEC-2014|        0|  -2972000|       0|  -7.772331E8|
|  ALBK|24-Dec-2014|24-DEC-2014|    -2000|  -2972000|    2000|  -7.773014E8|
|  ALBK|24-Dec-2014|24-DEC-2014|        0|  -2972000|    4000|  -7.773457E8|
|  ALBK|24-Dec-2014|24-DEC-2014|        0|  -2972000|    2000|  -7.772931E8|
|  ALBK|24-Dec-2014|24-DEC-2014|        0|  -2972000|    4000|  -7.773095E8|
|  ALBK|24-Dec-2014|24-DEC-2014|   -10000|  -2972000|   24000|  -7.775811E8|